# Finetuning MAMMAL Model for BBBP Prediction Task with LoRA 

In [1]:
#Importing Libraries

import evaluate
from scipy.special import softmax
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score,matthews_corrcoef
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from datasets import Dataset
from peft import LoraConfig, get_peft_model
import torch
import torch.nn.functional as F
from sklearn.preprocessing import LabelEncoder
from transformers import Trainer
from transformers import TrainingArguments, EarlyStoppingCallback
import pandas as pd
import numpy as np 

/storage/qnap_home/raghvendra2/micromamba/envs/Molformer/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from mammal.examples.molnet.molnet_infer import load_model, task_infer


task_dict = load_model(task_name="BBBP", device="cpu")
model = task_dict["model"]



In [ ]:
model.config

In [ ]:
## loading datasets

import pandas as pd

train_bbbp=pd.read_csv('/home/raghvendra2/Molformer_Finetuning/BBBP_Prediction_Molformer_Finetuned/bbbp/train.csv')
val_bbbp=pd.read_csv('/home/raghvendra2/Molformer_Finetuning/BBBP_Prediction_Molformer_Finetuned/bbbp/valid.csv')

Preparing Training and Validation Dataset

In [ ]:
smiles_list= train_bbbp['smiles'].tolist()
smiles_val=val_bbbp['smiles'].tolist()
train_tokenized=tokenizer(smiles_list)
val_tokenized=tokenizer(smiles_val)

In [ ]:
from datasets import Dataset
train_dataset = Dataset.from_dict(train_tokenized)
val_dataset = Dataset.from_dict(val_tokenized)

In [ ]:
train_labels = train_bbbp['p_np'].tolist() # Assuming tasks start from column 1
val_labels = val_bbbp['p_np'].tolist()

In [ ]:
train_dataset = train_dataset.add_column("labels", train_labels)
val_dataset = val_dataset.add_column("labels", val_labels)

## Hyperparameter tuning using WandB

In [ ]:
wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [ ]:
import wandb
from transformers import Trainer, TrainingArguments, EarlyStoppingCallback
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from evaluate import load
from datasets import Dataset
import numpy as np
import pandas as pd
import os
from scipy.special import softmax
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score,matthews_corrcoef

In [ ]:
def data_load():
    train_bbbp=pd.read_csv('/home/raghvendra2/Molformer_Finetuning/BBBP_Prediction_Task/bbbp/train.csv')
    val_bbbp=pd.read_csv('/home/raghvendra2/Molformer_Finetuning/BBBP_Prediction_Task/bbbp/valid.csv')

    return train_bbbp, val_bbbp

In [ ]:
def data_prep(data_process,tokenizer):

    smiles_list = data_process['smiles'].tolist()
    tokenized=tokenizer(smiles_list)
    
    
    dataset = Dataset.from_dict(tokenized)
    

    labels = data_process['p_np'].tolist() # Assuming tasks start from column 1
    
    dataset = dataset.add_column("labels", labels)
    

    return dataset

In [ ]:
from peft import LoraConfig, get_peft_model, PeftModel

def lora_config(r,lora_alpha,dropout):

    lora_config = LoraConfig(
        task_type="SEQ_CLS",  # Sequence classification task
        r=r,  # Rank of LoRA matrices
        lora_alpha=lora_alpha,  # Scaling factor double of rank( from the rule of thumb)
        target_modules='all-linear',
        lora_dropout=dropout  # Dropout rate
        #init_lora_weights="gaussian"
    )

    return lora_config

### Weighted Loss Trainer

In [ ]:
import torch

class_weights= [1-(train_dataset['labels'].count(0)/len(train_dataset['labels'])),
                           1-(train_dataset['labels'].count(1)/len(train_dataset['labels']))]

class_weights = torch.from_numpy(np.array(class_weights)).float().to("cuda")

class WeightedLossTrainer(Trainer):

    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):

        outputs = model(**inputs)
        logits = outputs.get("logits")

        # Extract labels
        labels = inputs.get("labels")

        # compute custom loss (suppose one has 2 labels with different weights)
        loss_func = torch.nn.CrossEntropyLoss(weight=class_weights)

        # compute loss
        loss = loss_func(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

NameError: name 'train_dataset' is not defined

### Focal Loss Trainer

In [ ]:
#focal loss computation
import torch.nn.functional as F
import torch

def focal_loss_multiclass(inputs, targets, alpha=1, gamma=2):
    log_prob = F.log_softmax(inputs, dim=-1)
    prob = torch.exp(log_prob)  # Convert log probabilities back to normal probabilities

    targets_one_hot = F.one_hot(targets, num_classes=inputs.shape[-1])
    pt = torch.sum(prob * targets_one_hot, dim=-1)  # Get probability of the true class

    focal_loss = -alpha * (1 - pt) ** gamma * torch.sum(log_prob * targets_one_hot, dim=-1)
    
    return focal_loss.mean() 


class FocalLossTrainer(Trainer):
    
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits

        loss = focal_loss_multiclass(logits, labels)
        
        return (loss, outputs) if return_outputs else loss 

In [ ]:
from evaluate import load
import numpy as np
from scipy.special import softmax
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score,matthews_corrcoef

accuracy_metric = load("accuracy")
mcc_metric= load("matthews_correlation")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    
    probabilities = softmax(logits, axis=1)[:, 1]  # Get probabilities for class 1
    predictions = np.argmax(logits, axis=1)  # Choose the most likely class
    

    mcc = matthews_corrcoef(labels, predictions)

    return {
        "eval_mcc_metric": mcc,
        "Accuracy": accuracy_metric.compute(predictions=predictions, references=labels)["accuracy"],
        "AUC-ROC": roc_auc_score(labels, probabilities),  # AUC-ROC requires probabilities
        "Precision": precision_score(labels, predictions),
        "Recall": recall_score(labels, predictions),
        "F1-score": f1_score(labels, predictions)
    } 



### Weighted Loss Trainer

In [ ]:
!pip install -q git+https://github.com/BiomedSciAI/biomed-multi-alignment.git

In [ ]:
from fuse.data.tokenizers.modular_tokenizer.op import ModularTokenizerOp
from mammal.model import Mammal


In [3]:
model_clin = AutoModelForSequenceClassification.from_pretrained(
        "ibm/biomed.omics.bl.sm.ma-ted-458m",
        num_labels=2,
        problem_type="single_label_classification",    
        trust_remote_code=True
    )
    

ValueError: Unrecognized model in ibm/biomed.omics.bl.sm.ma-ted-458m. Should have a `model_type` key in its config.json, or contain one of the following strings in its name: albert, align, altclip, aria, aria_text, audio-spectrogram-transformer, autoformer, aya_vision, bamba, bark, bart, beit, bert, bert-generation, big_bird, bigbird_pegasus, biogpt, bit, blenderbot, blenderbot-small, blip, blip-2, bloom, bridgetower, bros, camembert, canine, chameleon, chinese_clip, chinese_clip_vision_model, clap, clip, clip_text_model, clip_vision_model, clipseg, clvp, code_llama, codegen, cohere, cohere2, colpali, conditional_detr, convbert, convnext, convnextv2, cpmant, ctrl, cvt, dab-detr, dac, data2vec-audio, data2vec-text, data2vec-vision, dbrx, deberta, deberta-v2, decision_transformer, deformable_detr, deit, depth_anything, depth_pro, deta, detr, diffllama, dinat, dinov2, dinov2_with_registers, distilbert, donut-swin, dpr, dpt, efficientformer, efficientnet, electra, emu3, encodec, encoder-de

In [ ]:

# Initialize W&B with sweep
def run_training():
    run = wandb.init(project="BBBP MAMMAL Hyperparameter Tuning")
    config = run.config   

    # Define unique save path for each W&B run
    save_dir = f"./models_MAMMAL_BBBP/{wandb.run.id}"  # Unique directory for each run
    os.makedirs(save_dir, exist_ok=True)


    model_clin = AutoModelForSequenceClassification.from_pretrained(
        "ibm/biomed.omics.bl.sm.ma-ted-458m",
        num_labels=2,
        problem_type="single_label_classification",    
        trust_remote_code=True
    )
    tokenizer = ModularTokenizerOp.from_pretrained("ibm/biomed.omics.bl.sm.ma-ted-458m")

    

    # Load data
    train_data, val_data = data_load()
    training_data = data_prep(train_data, tokenizer)
    validation_data = data_prep(val_data, tokenizer)


    # Apply LoRA
    peft_config = lora_config(config.r, config.lora_alpha, config.dropout)
    lora_model = get_peft_model(model_clin, peft_config)
    
    lora_model.print_trainable_parameters()

    # Training arguments
    training_args = TrainingArguments(
        output_dir=save_dir,
        evaluation_strategy="epoch",
        learning_rate=config.lr,
        per_device_train_batch_size=32,
        per_device_eval_batch_size=32,
        num_train_epochs=20,
        weight_decay=0.01,
        save_strategy="epoch",  # Save model at each epoch
        logging_dir=f"./logs_BBBP/{wandb.run.id}",
        logging_strategy="steps",
        logging_steps=100,
        report_to="wandb",
        save_total_limit=1,
        load_best_model_at_end=True,
        metric_for_best_model="eval_mcc_metric"
    )

    accuracy_metric = load("accuracy")

    def compute_metrics(eval_pred):
        logits, labels = eval_pred
        probabilities = softmax(logits, axis=1)[:, 1]  
        predictions = np.argmax(logits, axis=1)  
        mcc = matthews_corrcoef(labels, predictions)

        return {
            "eval_mcc_metric": mcc,
            "Accuracy": accuracy_metric.compute(predictions=predictions, references=labels)["accuracy"],
            "AUC-ROC": roc_auc_score(labels, probabilities),
            "Precision": precision_score(labels, predictions),
            "Recall": recall_score(labels, predictions),
            "F1-score": f1_score(labels, predictions)
        }

    # Initialize trainer
    trainer = WeightedLossTrainer(
        model=lora_model,
        args=training_args,
        train_dataset=training_data,
        eval_dataset=validation_data,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=5)]
    )

    # Train model
    trainer.train()

    # Save model and tokenizer for this run
    trainer.save_model(save_dir)
    
    print(f"Model saved to {save_dir}")
    
    wandb.finish()


### Focal loss trainer

In [ ]:


# Initialize W&B with sweep
def run_training():
    run = wandb.init(project="BBBP focal loss Hyperparameter Tuning")
    config = run.config   

    # Define unique save path for each W&B run
    save_dir = f"./models_BBBP_chemberta_focal_loss/{wandb.run.id}"  # Unique directory for each run
    os.makedirs(save_dir, exist_ok=True)

    tokenizer = AutoTokenizer.from_pretrained("DeepChem/ChemBERTa-77M-MTR")
    model = AutoModelForSequenceClassification.from_pretrained(
    "DeepChem/ChemBERTa-77M-MTR",                                 # Define any other model here
    num_labels=2,
    problem_type="single_label_classification",
    trust_remote_code=True
    )
    

    # Load data
    train_data, val_data = data_load()
    training_data = data_prep(train_data, tokenizer)
    validation_data = data_prep(val_data, tokenizer)


    # Apply LoRA
    peft_config = lora_config(config.r, config.lora_alpha, config.dropout)
    lora_model = get_peft_model(model, peft_config)
    
    lora_model.print_trainable_parameters()

    # Training arguments
    training_args = TrainingArguments(
        output_dir=save_dir,
        evaluation_strategy="epoch",
        learning_rate=config.lr,
        per_device_train_batch_size=32,
        per_device_eval_batch_size=32,
        num_train_epochs=20,
        weight_decay=0.01,
        save_strategy="epoch",  # Save model at each epoch
        logging_dir=f"./logs_BBBP/{wandb.run.id}",
        logging_strategy="steps",
        logging_steps=100,
        report_to="wandb",
        save_total_limit=10,
        load_best_model_at_end=True,
        metric_for_best_model="eval_mcc_metric"
    )

    accuracy_metric = load("accuracy")

    def compute_metrics(eval_pred):
        logits, labels = eval_pred
        probabilities = softmax(logits, axis=1)[:, 1]  
        predictions = np.argmax(logits, axis=1)  
        mcc = matthews_corrcoef(labels, predictions)

        return {
            "eval_mcc_metric": mcc,
            "Accuracy": accuracy_metric.compute(predictions=predictions, references=labels)["accuracy"],
            "AUC-ROC": roc_auc_score(labels, probabilities),
            "Precision": precision_score(labels, predictions),
            "Recall": recall_score(labels, predictions),
            "F1-score": f1_score(labels, predictions)
        }

    # Initialize trainer
    trainer = FocalLossTrainer(
        model=lora_model,
        args=training_args,
        train_dataset=training_data,
        eval_dataset=validation_data,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=10)]
    )

    # Train model
    trainer.train()

    # Save model and tokenizer for this run
    trainer.save_model(save_dir)
    
    print(f"Model saved to {save_dir}")
    
    wandb.finish()


In [ ]:
def main():

    sweep_config = {
    "name": "BBBP Chemberta Hyperparameter Tuning",
    "method": "bayes",
    "metric": {
        "goal": "maximize", 
        "name": "eval/mcc_metric"
    },
    "parameters": {
        "lr": {
            "distribution": "uniform",
            "min": 1e-5,  
            "max": 2e-5
        },
        "r": {
            "values": [8,16,32,64, 128]
        },
        "lora_alpha": {
            "values": [8,16,32,64,128]
        },
        "dropout": {
            "values": [0.0,0.1,0.2]
        },
        
        "optimizer": {
            "value": ["adamw"]
        }
    }
}

    sweep_id = wandb.sweep(sweep_config, project="huggingface")
    wandb.agent(sweep_id, function=run_training, count=10)

    api = wandb.Api()
    sweep = api.sweep(f"huggingface/{sweep_id}")
    print(sweep.runs[0].summary_metrics)

    runs_with_rmse = [run for run in sweep.runs if 'eval/mcc_metric' in run.summary_metrics]
    if runs_with_rmse:
        # Sort by rmse in descending order (maximize)
        best_run = sorted(runs_with_rmse, key=lambda run: run.summary_metrics['eval/mcc_metric'])[0]
    else:
        raise ValueError("No runs found with 'eval/mcc_metric' metric.")

    best_hyperparameters = best_run.config
    print(f"Best hyperparameters: {best_hyperparameters}")
    

if __name__ == "__main__":
    os.environ["CUDA_VISIBLE_DEVICES"] = "0"
    main()

Create sweep with ID: s602kzmr
Sweep URL: https://wandb.ai/harodharsha21-iit-ropar/huggingface/sweeps/s602kzmr


wandb: Agent Starting Run: y1eq8vgp with config:
wandb: 	dropout: 0.1
wandb: 	lora_alpha: 16
wandb: 	lr: 1.3111835528238495e-05
wandb: 	optimizer: ['adamw']
wandb: 	r: 128


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at DeepChem/ChemBERTa-77M-MTR and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/storage/qnap_home/raghvendra2/micromamba/envs/Molformer/lib/python3.10/site-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


trainable params: 2,127,234 || all params: 5,703,156 || trainable%: 37.2992


/tmp/ipykernel_1781731/2934716305.py:68: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `FocalLossTrainer.__init__`. Use `processing_class` instead.
  trainer = FocalLossTrainer(
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


...

wandb: Agent Starting Run: p4hh23r5 with config:
wandb: 	dropout: 0
wandb: 	lora_alpha: 128
wandb: 	lr: 1.8067884774620412e-05
wandb: 	optimizer: ['adamw']
wandb: 	r: 16


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at DeepChem/ChemBERTa-77M-MTR and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/storage/qnap_home/raghvendra2/micromamba/envs/Molformer/lib/python3.10/site-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


trainable params: 395,938 || all params: 3,842,612 || trainable%: 10.3039


/tmp/ipykernel_1781731/2934716305.py:68: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `FocalLossTrainer.__init__`. Use `processing_class` instead.
  trainer = FocalLossTrainer(
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Epoch,Training Loss,Validation Loss,Mcc Metric,Accuracy,Auc-roc,Precision,Recall,F1-score
1,No log,0.139675,0.174226,0.721951,0.858161,0.722222,0.986207,0.833819
2,0.137100,0.111588,0.227441,0.731707,0.907126,0.731959,0.979310,0.837758
3,0.137100,0.100113,0.510343,0.809756,0.922874,0.797753,0.979310,0.879257
4,0.081200,0.092847,0.524643,0.814634,0.924828,0.802260,0.979310,0.881988
5,0.081200,0.091928,0.564238,0.829268,0.923218,0.819767,0.972414,0.889590
6,0.066100,0.094641,0.591400,0.839024,0.918966,0.829412,0.972414,0.895238
7,0.066100,0.097466,0.591400,0.839024,0.920115,0.829412,0.972414,0.895238
8,0.057100,0.097571,0.618043,0.848780,0.920690,0.839286,0.972414,0.900958
9,0.057100,0.099037,0.631194,0.853659,0.921724,0.844311,0.972414,0.903846
